## Imports and Setup

In [ ]:
import os
import pprint
os.chdir("..")

from src.dfcv_colocation_mapping import data_download
from src.dfcv_colocation_mapping import data_utils
from src.dfcv_colocation_mapping import map_utils

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

import warnings
warnings.filterwarnings(action="ignore", message=r"datetime.datetime.utcnow")
warnings.simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

%load_ext autoreload
%autoreload 2

## Parameters

In [33]:
iso_code = "DJI"
adm_level = "ADM2"

## Download Datasets

In [34]:
dm = data_download.DatasetManager(
    iso_code, 
    adm_level=adm_level,
    data_dir="data/",
    config_file="src/dfcv_colocation_mapping/configs/data_config.yaml",
    acled_file="creds/acled_creds.yaml",
    dtm_file="creds/dtm_creds.yaml",
    idmc_file="creds/idmc_creds.yaml",
    adm_config_file="src/dfcv_colocation_mapping/configs/adm_config.yaml",
    osm_config_file="src/dfcv_colocation_mapping/configs/osm_config.yaml",
    mhs_aggregation="power_mean",
    resample_worldcover=True
)

In [35]:
dm.set_acled_filters(
    category="disorder_type", 
    values = [
        'Strategic developments'
    ],
    operation="exclude"
)
dm.set_acled_filters(
    category="sub_event_type", 
    values = [
        'Peaceful protest'
    ],
    operation="exclude"
)

# Sense-check inclusion/exclusion criteria
pprint.pprint(dm.acled_filters)

{'exclude': {'disorder_type': ['Strategic developments'],
             'event_type': ['Strategic developments'],
             'sub_event_type': ['Agreement',
                                'Arrests',
                                'Change to group/activity',
                                'Disrupted weapons use',
                                'Headquarters or base established',
                                'Looting/property destruction',
                                'Non-violent transfer of territory',
                                'Other',
                                'Peaceful protest']},
 'include': {'disorder_type': ['Political violence',
                               'Political violence; Demonstrations',
                               'Demonstrations'],
             'event_type': ['Protests',
                            'Violence against civilians',
                            'Battles',
                            'Riots',
                            'Explosions/

In [ ]:
dm.download_all()
geoplot = map_utils.GeoPlot(dm, data_dir="data/", map_config_file="src/dfcv_colocation_mapping/configs/map_config.yaml")
dm.data.sort_values("mhs_all_wbg_acled_worldpop_exposure_relative", ascending=False).head(3)

## AHP Weight Calculation [Optional]

In [ ]:
dm.calculate_ahp()
geoplot = map_utils.GeoPlot(dm)
dm.data.sort_values(f"mhs_all_wbg_acled_{dm.asset}_exposure_relative", ascending=False).head(3)

## Plot Regional Groupings

In [ ]:
ax, x = geoplot.plot_geoboundaries(
    adm_level="ADM2",
    group="ADM1"
);

## Plot Population

In [ ]:
ax, x = geoplot.plot_choropleth( 
    var="dtm_idp_total",
    #kwargs={"legend_type": "colorbar"} # "legend_shrink":0.125}
)

In [ ]:
ax, x = geoplot.plot_choropleth( 
    var="bem",
    kwargs={"legend_type": "colorbar"},
    zoom_to = {"ADM1": "Western"}
)

In [ ]:
ax, x = geoplot.plot_choropleth( 
    var="worldcover_builtup",
    kwargs={"legend_type": "colorbar"}
    #kwargs={"legend_type": "colorbar", "legend_shrink":0.125}
)

In [ ]:
ax, x = geoplot.plot_choropleth( 
    var="acled_conflict_count",
    kwargs={
        "legend_type": "colorbar",
        "cmap": "Reds",
        "edgecolor": "dimgray",
    }
);

In [ ]:
ax, x = geoplot.plot_choropleth( 
    var="numPresentIdpInd",
    data=dm.dtm
)

In [ ]:
ax, x = geoplot.plot_choropleth( 
    var="acled_conflict_count",
    #nbins=3,
    #binning="quantiles",
    #zoom_to = {"ADM1": "Dakar"}
);

In [ ]:
geoplot.plot_points(
    "event_type", 
    dataset="acled", 
    clustering=False,  
    kwargs={
        "alpha": 1,
        #"legend_y": 0.435,
        #"legend_x": 0.275,
        "markerscale": 2,
        "cmap": "tab10"
    },
    #zoom_to=zoom_to,
    #ax=ax, xpos=x,
)

## Plot Conflict Exposure - ACLED (Population Best)

In [ ]:
source = "wbg_acled"
geoplot.plot_choropleth( 
    var=f"{source}_worldpop_exposure",
    subtitle="By township, from 2021 to 2025",
    #binning="quantiles"
    #kwargs={"legend_type": "colorbar"}
)
geoplot.plot_choropleth( 
    var=f"{source}_worldpop_exposure_relative",
    subtitle="By township, from 2021 to 2025",
    #nbins=6,
    #var_bounds=[0,100]
    kwargs={"legend_type": "colorbar"}
);

In [ ]:
source = "wbg_acled"
geoplot.plot_choropleth( 
    var=f"{source}_worldcover_builtup_exposure",
    subtitle="By township, from 2021 to 2025",
    #kwargs={"legend_type": "colorbar"}
)
geoplot.plot_choropleth( 
    var=f"{source}_worldcover_builtup_exposure_relative",
    subtitle="By township, from 2021 to 2025",
    kwargs={"legend_type": "colorbar"}
);

## Plot Conflict Exposure - ACLED (Custom Calculation)

In [ ]:
source = "wbg_acled"
geoplot.plot_choropleth( 
    var=f"{source}_worldcover_builtup_exposure",
    #binning="quantiles"
)
ax, x = geoplot.plot_choropleth( 
    var=f"{source}_bem_exposure",
    #var_bounds=[0, 100],
);

## Plot Conflict Exposure - UCDP

In [ ]:
source = "ucdp"
geoplot.plot_choropleth( 
    var=f"{source}_bem_exposure"
)
geoplot.plot_choropleth( 
    var=f"{source}_bem_exposure_relative",
    var_bounds=[0, 100]
);

## Zoom Choropleth to a specific region

In [ ]:
source = "wbg_acled"
geoplot.plot_choropleth( 
    var=f"{source}_{dm.asset}_exposure",
    zoom_to={"ADM1": "Lomami"},
    subtitle="By township, from 2021 to 2025"
)

In [ ]:
ax = geoplot.plot_choropleth( 
    var=f"{source}_{dm.asset}_exposure_relative",
    zoom_to={"ADM1": "Lakes"},
    subtitle="By township, from 2021 to 2025",
    var_bounds=[0, 100],
    kwargs={
        'legend_shrink': 0.20,
        'legend_type': 'colorbar',
    },
);

In [ ]:
ax = geoplot.plot_choropleth( 
    var=f"numPresentIdpInd",
    subtitle="By township, from 2021 to 2025",
    #var_bounds=[0, 100],
    kwargs={
        'legend_shrink': 0.20,
        'legend_type': 'colorbar',
    },
);

In [ ]:
source = "wbg_acled"
geoplot.plot_choropleth( 
    var=f"{source}_{dm.asset}_exposure",
    zoom_to={"ADM1": "Eastern Province"},
    subtitle="By township, from 2021 to 2025"
)
ax = geoplot.plot_choropleth( 
    var=f"{source}_{dm.asset}_exposure_relative",
    zoom_to={"ADM1": "Eastern Province"},
    subtitle="By township, from 2021 to 2025",
    var_bounds=[0, 100],
);

## Map Exposure to a Single Hazard

In [ ]:
hazard = "landslide_earthquake"
geoplot.plot_raster(hazard);

In [ ]:
geoplot.plot_choropleth(f"{hazard}_worldpop_exposure_relative", kwargs={"legend_type": "colorbar"});

In [ ]:
geoplot.plot_choropleth(f"{hazard}_worldcover_cropland_exposure");

In [ ]:
geoplot.plot_choropleth(f"{hazard}_worldcover_cropland_exposure_relative");

In [ ]:
geoplot.plot_choropleth(
    f"{hazard}_bem_exposure",
    #var_bounds=[0, 100],
    kwargs={"legend_type": "colorbar"},
);

In [ ]:
geoplot.plot_choropleth(
    f"{hazard}_worldpop_intensity_weighted_exposure_relative",
);

## Plot Multi-hazard Exposure Index and Multi-hazard Conflict Exposure Index

In [ ]:
geoplot.plot_choropleth( 
    var=f"mhs_all_wbg_acled_worldpop_exposure_relative",
    kwargs={
        "legend_type": "colorbar", 
        "linewidth": 0.1
    }
);

In [ ]:
geoplot.plot_choropleth( 
    var=f"mhs_all_wbg_acled_worldpop_exposure_relative",
    kwargs={
        "legend_type": "colorbar", 
        "linewidth": 0.1
    }
);

In [ ]:
ax, x = geoplot.plot_choropleth( 
    var=f"mhs_all_worldcover_cropland_exposure_relative",
    kwargs={"legend_type": "colorbar", "legend_y": 0.5,}
);
geoplot.plot_points(
    "type_of_violence", 
    dataset="acled", 
    clustering=False,  
    kwargs={
        "alpha": 0.5,
        "legend_y": 0.425,
        "markerscale": 0.5,
        "cmap": "tab10"
    },
    ax=ax, xpos=x,
);

In [ ]:
geoplot.plot_choropleth( 
    var=f"mhs_all_wbg_acled_worldpop_exposure_relative",
    subtitle="By township, from 2021 to 2025",
    kwargs={"legend_type": "colorbar"}
);

In [ ]:
geoplot.plot_choropleth( 
    var=f"mhs_all_wbg_acled_bem_exposure_relative",
    kwargs={"legend_type": "colorbar"}
);

# Bivariate Choropleth: Overlay Multi-hazard and Conflict Exposure

In [ ]:
geoplot.plot_bivariate_choropleth( 
    var1=f"wbg_acled_worldpop_exposure_relative",
    var2=f"mhs_all_worldpop_exposure_relative",
    var1_bounds=[0, 1],
    var2_bounds=[0, 1],
    binning="equal_intervals",
    kwargs={
        "legend_fontsize": 4, 
        "edgecolor": "white", 
        "linewidth": 0.2
    }
);


In [ ]:
geoplot.plot_bivariate_choropleth( 
    var2=f"idmc_disaster_idp_total",
    var1=f"mhs_all_worldpop_exposure_relative",
    #var1_bounds=[0, 1],
    #var2_bounds=[0, 1],
    binning="equal_intervals",
    kwargs={
        "legend_fontsize": 4, 
        "edgecolor": "white", 
        "linewidth": 0.2
    }
);

In [ ]:
geoplot.plot_bivariate_choropleth( 
    var1=f"wbg_acled_worldcover_builtup_exposure_relative",
    var2=f"mhs_all_worldcover_builtup_exposure_relative",
    #var1_bounds=[0, 1],
    #var2_bounds=[0, 1],
    binning="equal_intervals",
    kwargs={
        "legend_fontsize": 4, 
        "edgecolor": "white", 
        "linewidth": 0.2
    }
);

In [ ]:
geoplot.plot_bivariate_choropleth( 
    var1=f"wbg_acled_bem_exposure_relative",
    var2=f"mhs_all_bem_exposure_relative",
    #var1_bounds=[0, 1],
    #var2_bounds=[0, 1],
    binning="equal_intervals",
    kwargs={
        "legend_fontsize": 4, 
        "edgecolor": "white", 
        "linewidth": 0.2
    }
);

## Interactive Map

In [ ]:
m = geoplot.plot_folium(
    adm_level=adm_level,
    var="worldcover_builtup",
    kwargs={"zoom_start": 9}
)
m

In [ ]:
 m.save("folium.html")